In [14]:
# Global keys

_BATCH_ACCOUNT_NAME = "climatebasedbatch"
_BATCH_ACCOUNT_KEY = "W94ukoxG2neFkk6teOVZ3IQ8IQjmPJqPcFq48I9lLzCrPEQSRFS/+euaUEkkSyPoulUgnx5IEZxztA9574Hluw=="
_BATCH_ACCOUNT_URL = "https://climatebasedbatch.westeurope.batch.azure.com"

_STORAGE_ACCOUNT_NAME = "radfiles"
_STORAGE_ACCOUNT_KEY = "aRRVzOkO/kwS35CIwNVIa18aGoMfZD5D3yAy3GlorkkU2G+9q5rAscXoC21IIylJZerBefwMgxYYF3qzquALrw=="

_POOL_ID = "testpool"
_MIN_POOL_NODE = 1
_MAX_POOL_NODE = 100

_POOL_VM_SIZE = 'BASIC_A1'
_NODE_OS_PUBLISHER = 'Canonical'
_NODE_OS_OFFER = 'UbuntuServer'
_NODE_OS_SKU = '16'

_JOB_NAME = "0000000-testjob-3513"

_JOB_DIRECTORY = "./radfiles"

_COPY_TO_BLOB_LOCAL = "./docker/copy_to_blob.py"

In [3]:
# Imports

import azure.storage.blob as azureblob
import azure.batch.models as batchmodels
import azure.batch.batch_service_client as batch
import azure.batch.batch_auth as batchauth
import datetime
import os

In [7]:
# Create the blob client - for use in obtaining referecnes to the blob stroage containers and uploading files to those containers
blob_client = azureblob.BlockBlobService(account_name=_STORAGE_ACCOUNT_NAME, account_key=_STORAGE_ACCOUNT_KEY)
print("Blob client generated:  {0:}".format(blob_client))

# Generate shard key credentials enabling transaction with the batch client
credentials = batchauth.SharedKeyCredentials(_BATCH_ACCOUNT_NAME, _BATCH_ACCOUNT_KEY)
print("Credentials generated:  {0:}".format(credentials))

# Create the batch client - for use in transacting with batch nodes
batch_client = batch.BatchServiceClient(credentials, base_url=_BATCH_ACCOUNT_URL)
print("Batch client generated: {0:}".format(batch_client))

Blob client generated:  <azure.storage.blob.blockblobservice.BlockBlobService object at 0x10d202f28>
Credentials generated:  <azure.batch.batch_auth.SharedKeyCredentials object at 0x10d1526d8>
Batch client generated: <azure.batch.batch_service_client.BatchServiceClient object at 0x10d2190f0>


In [27]:
# KILL ALL PROCESSES! - TIDY UP AND NEVER LOOK BACK!

# containers
# blob_client.delete_container(_JOB_NAME)
# pools
batch_client.pool.delete(_POOL_ID)
# jobs
# for _JOB_ID in jobs:
#     batch_client.job.delete(_JOB_ID)

In [10]:
# Create container for job file storage
blob_client.create_container(_JOB_NAME, fail_on_exist=False)
print("Container created: [{0:}]".format(_JOB_NAME))

Container created: [0000000-testjob-3513]


In [19]:
# Upload sky matrix and surfaces files into job container

def directory_files(directory):
    """
    Generates a list of the files within a directory
    :param str directory: A path to a directory.
    :return list: List of files
    """
    return [item for sublist in [[os.path.join(path, name) for name in files] for path, subdirs, files in os.walk(directory)] for item in sublist]

def upload_file_to_container(block_blob_client, container_name, file_path):
    """
    Uploads a local file to an Azure Blob storage container.
    :param block_blob_client: An Azure blockblobservice client.
    :type block_blob_client: `azure.storage.blob.BlockBlobService`
    :param str container_name: The name of the Azure Blob storage container.
    :param str file_path: The local path to the file.
    :rtype: `azure.batch.models.ResourceFile`
    :return: A ResourceFile initialized with a SAS URL appropriate for Batch tasks.
    """

    blob_name = os.path.basename(file_path)
    print('Uploading file {0:} to container [{1:}/{0:}]'.format(blob_name, container_name))
    block_blob_client.create_blob_from_path(container_name, blob_name, file_path)
    sas_token = block_blob_client.generate_blob_shared_access_signature(container_name, blob_name, permission=azureblob.BlobPermissions.READ, expiry=datetime.datetime.utcnow() + datetime.timedelta(hours=24))
    sas_url = block_blob_client.make_blob_url(container_name, blob_name, sas_token=sas_token)
    
    return batchmodels.ResourceFile(file_path=blob_name, blob_source=sas_url)

# Upload the files common to all simulations (Sky Matrix, Context Surfaces and _APP_FILEs)
_SKY_MTX = upload_file_to_container(blob_client, _JOB_NAME, os.path.join(_JOB_DIRECTORY, "sky_mtx.json"))
_SURFACES = upload_file_to_container(blob_client, _JOB_NAME, os.path.join(_JOB_DIRECTORY, "surfaces.json"))
_COPY_TO_BLOB = upload_file_to_container(blob_client, _JOB_NAME, _COPY_TO_BLOB_LOCAL)

# Upload the files unique to each simulation (individual analysis grids)
_ANALYSIS_GRIDS = []
for _file in directory_files(os.path.join(_JOB_DIRECTORY, "AnalysisGrids")):
    _ANALYSIS_GRIDS.append(upload_file_to_container(blob_client, _JOB_NAME, _file))

Uploading file sky_mtx.json to container [0000000-testjob-3513/sky_mtx.json]
Uploading file surfaces.json to container [0000000-testjob-3513/surfaces.json]
Uploading file copy_to_blob.py to container [0000000-testjob-3513/copy_to_blob.py]
Uploading file zone4.json to container [0000000-testjob-3513/zone4.json]
Uploading file zone3.json to container [0000000-testjob-3513/zone3.json]
Uploading file zone2.json to container [0000000-testjob-3513/zone2.json]
Uploading file zone1.json to container [0000000-testjob-3513/zone1.json]


In [6]:
# Get shared access signature providing write access to the container
def get_container_sas_token(block_blob_client, container_name, blob_permissions):
    """
    Obtains a shared access signature granting the specified permissions to the
    container.
    :param block_blob_client: A blob service client.
    :type block_blob_client: `azure.storage.blob.BlockBlobService`
    :param str container_name: The name of the Azure Blob storage container.
    :param BlobPermissions blob_permissions:
    :rtype: str
    :return: A SAS token granting the specified permissions to the container.
    """

    container_sas_token = block_blob_client.generate_container_shared_access_signature(
        container_name,
        permission=blob_permissions,
        expiry=datetime.datetime.utcnow() + datetime.timedelta(hours=24))

    return container_sas_token

output_container_sas_token = get_container_sas_token(blob_client, _JOB_NAME, azureblob.BlobPermissions.WRITE)
print(output_container_sas_token)

sr=c&sp=w&sig=8NsHFKdc3Ly7HTEKpcW0X1Se3Slo5vY3HZRZNN%2BXtN8%3D&sv=2018-03-28&se=2018-08-19T21%3A54%3A13Z


In [26]:
# Create the pool containing the compute nodes executing the tasks
def select_latest_verified_vm_image_with_node_agent_sku(batch_client, publisher, offer, sku_starts_with):
    """Select the latest verified image that Azure Batch supports given a publisher, offer and sku (starts with filter).
    :param batch_client: The batch client to use.
    :type batch_client: `batchserviceclient.BatchServiceClient`
    :param str publisher: vm image publisher
    :param str offer: vm image offer
    :param str sku_starts_with: vm sku starts with filter
    :rtype: tuple
    :return: (node agent sku id to use, vm image ref to use)
    """
    # get verified vm image list and node agent sku ids from service
    node_agent_skus = batch_client.account.list_node_agent_skus()
    # pick the latest supported sku
    skus_to_use = [
        (sku, image_ref) for sku in node_agent_skus for image_ref in sorted(
            sku.verified_image_references, key=lambda item: item.sku)
        if image_ref.publisher.lower() == publisher.lower() and
        image_ref.offer.lower() == offer.lower() and
        image_ref.sku.startswith(sku_starts_with)
    ]
    # skus are listed in reverse order, pick first for latest
    sku_to_use, image_ref_to_use = skus_to_use[0]
    return (sku_to_use.id, image_ref_to_use)

def wrap_commands_in_shell(ostype, commands):
    """Wrap commands in a shell
    :param list commands: list of commands to wrap
    :param str ostype: OS type, linux or windows
    :rtype: str
    :return: a shell wrapping commands
    """
    if ostype.lower() == "linux":
        return "/bin/bash -c \"set -e; set -o pipefail; {}; wait\"".format(";".join(commands))
    elif ostype.lower() == "windows":
        return "cmd.exe /c {}".format("&".join(commands))
    else:
        raise ValueError("unknown ostype: {}".format(ostype))
        
def print_batch_exception(batch_exception):
    """
    Prints the contents of the specified Batch exception.
    :param batch_exception:
    """
    print('-------------------------------------------')
    print('Exception encountered:')
    if (batch_exception.error and batch_exception.error.message and
            batch_exception.error.message.value):
        print(batch_exception.error.message.value)
        if batch_exception.error.values:
            print()
            for mesg in batch_exception.error.values:
                print('{}:\t{}'.format(mesg.key, mesg.value))
    print('-------------------------------------------')

def create_pool(batch_service_client, pool_id, resource_files, publisher, offer, sku, node_count):
    """
    Creates a pool of compute nodes with the specified OS settings.

    :param batch_service_client: A Batch service client.
    :type batch_service_client: `azure.batch.BatchServiceClient`
    :param str pool_id: An ID for the new pool.
    :param list resource_files: A collection of resource files for the pool's start task.
    :param str publisher: Marketplace image publisher
    :param str offer: Marketplace image offer
    :param str sku: Marketplace image sku
    """
    print('Creating pool [{}]...'.format(pool_id))

    # Specify the commands for the pool's start task to be run on each node as it joins the pool.
    task_commands = [
        # Install pip
        "curl -fSsL https://bootstrap.pypa.io/get-pip.py | python",
        # Install the azure-storage module so that the task script can access Azure Blob storage
        "pip install azure-storage==0.32.0",
        # Install docker
        "sudo apt-get install docker -y && sudo apt-get install docker.io -y",
        # Pull RadHoneyWhale from docker hub
        "sudo docker pull tgerrish/bhrad"]

    # Get the node agent SKU and image reference for the virtual machine configuration.
    sku_to_use, image_ref_to_use = select_latest_verified_vm_image_with_node_agent_sku(batch_service_client, publisher, offer, sku)
    user = batchmodels.AutoUserSpecification(scope=batchmodels.AutoUserScope.pool, elevation_level=batchmodels.ElevationLevel.admin)
    new_pool = batch.models.PoolAddParameter(
        id=pool_id,
        virtual_machine_configuration=batchmodels.VirtualMachineConfiguration(
            image_reference=image_ref_to_use,
            node_agent_sku_id=sku_to_use),
        vm_size=_POOL_VM_SIZE,
        resize_timeout=datetime.timedelta(minutes=15),
        target_dedicated_nodes=_POOL_NODE_COUNT,
        start_task=batch.models.StartTask(
            command_line=wrap_commands_in_shell(
                "linux",
                task_commands),
            user_identity=batchmodels.UserIdentity(auto_user=user),
            wait_for_success=True,
            resource_files=resource_files),
    )

    try:
        batch_service_client.pool.add(new_pool)
    except batchmodels.batch_error.BatchErrorException as err:
        print_batch_exception(err)
        raise

_POOL_NODE_COUNT = len(_ANALYSIS_GRIDS)

pool = create_pool(batch_client, _POOL_ID, [_SKY_MTX, _SURFACES, _COPY_TO_BLOB], _NODE_OS_PUBLISHER, _NODE_OS_OFFER, _NODE_OS_SKU, _POOL_NODE_COUNT)

print("Pool created ...")

Creating pool [testpool]...
Pool created ...


In [11]:
# Add jobs to the pool

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

def create_job(batch_service_client, job_id, pool_id):
    """
    Creates a job with the specified ID, associated with the specified pool.

    :param batch_service_client: A Batch service client.
    :type batch_service_client: `azure.batch.BatchServiceClient`
    :param str job_id: The ID for the job.
    :param str pool_id: The ID for the pool.
    """
    print('Creating job [{0:}]...'.format(job_id))

    job = batch.models.JobAddParameter(job_id, batch.models.PoolInformation(pool_id=pool_id))

    try:
        batch_service_client.job.add(job)
    except batchmodels.batch_error.BatchErrorException as err:
        print_batch_exception(err)
        raise

def add_tasks(batch_service_client, job_id, input_files,
              input_file_names, output_container_name, output_container_sas_token, other_input_files):
    """
    Adds a task for each input file in the collection to the specified job.

    :param batch_service_client: A Batch service client.
    :type batch_service_client: `azure.batch.BatchServiceClient`
    :param str job_id: The ID of the job to which to add the tasks.
    :param list input_files: A collection of input files. One task will be
     created for each input file.
    :param output_container_name: The ID of an Azure Blob storage container to
    which the tasks will upload their results.
    :param output_container_sas_token: A SAS token granting write access to
    the specified Azure Blob storage container.
    """

    print('Adding {} tasks to job [{}]...'.format(len(input_files), job_id))

    user = batchmodels.UserIdentity(
    auto_user=batchmodels.AutoUserSpecification(
        elevation_level=batchmodels.ElevationLevel.admin,
        scope=batchmodels.AutoUserScope.task))

    tasks = list()

    for idx, input_file in enumerate(input_files):
        filename = input_file_names[idx]

        command = ["sudo bash",
                   "sudo docker run -v $AZ_BATCH_TASK_WORKING_DIR:$HOME/home -a stdout tgerrish/radtemp:latest python RunHoneybeeRadiance.py -p $HOME/home/zone1.json -sm $HOME/home/sky_mtx.json -s $HOME/home/surfaces.json",
                   "python $AZ_BATCH_NODE_SHARED_DIR/copy_to_blob.py --filepath {0:} --blobname {1:} --storageaccount {2:} --storagecontainer {3:} --sastoken '{4:}'".format(
                       "$AZ_BATCH_NODE_SHARED_DIR/{0:}".format(filename.replace(".json", "_result.json")),
                        filename.replace(".json", "_result.json"),
                       _STORAGE_ACCOUNT_NAME,
                       output_container_name,
                       output_container_sas_token
                   )]

        tasks.append(batch.models.TaskAddParameter(
                'Task_{}'.format(idx,filename),
                common.helpers.wrap_commands_in_shell('linux', command),
                resource_files=[input_file],
                user_identity=user
                )
        )
    batch_service_client.task.add_collection(job_id, tasks)

def wait_for_tasks_to_complete(batch_service_client, job_id, timeout):
    """
    Returns when all tasks in the specified job reach the Completed state.

    :param batch_service_client: A Batch service client.
    :type batch_service_client: `azure.batch.BatchServiceClient`
    :param str job_id: The id of the job whose tasks should be to monitored.
    :param timedelta timeout: The duration to wait for task completion. If all
    tasks in the specified job do not reach Completed state within this time
    period, an exception will be raised.
    """
    timeout_expiration = datetime.datetime.now() + timeout

    print("Monitoring all tasks for 'Completed' state, timeout in {}...".format(timeout), end='')

    while datetime.datetime.now() < timeout_expiration:
        print('.', end='')
        sys.stdout.flush()
        tasks = [batch_service_client.task.list(job) for job in job_id]

        tasks = [item for sublist in tasks for item in sublist]

        incomplete_tasks = [task for task in tasks if
                            task.state != batchmodels.TaskState.completed]
        if not incomplete_tasks:
            print()
            return True
        else:
            time.sleep(1)

    print()
    raise RuntimeError("ERROR: Tasks did not reach 'Completed' state within "
                       "timeout period of " + str(timeout))

jobs = list()
_JOB_ID = "radjob"

for i, (files, names) in enumerate(zip(list(chunks(grid_files, 100)),list(chunks(grid_file_names, 100)))):
    job_id = _JOB_ID + "_" + str(i)
    jobs.append(job_id)

    # Create the job that will run the tasks.
    create_job(batch_client, job_id, _POOL_ID)

    # Add the tasks to the job. We need to supply a container shared access
    # signature (SAS) token for the tasks so that they can upload their output
    # to Azure Storage.

    add_tasks(batch_client,
              job_id,
              files,
              names,
              _JOB_NAME,
              output_container_sas_token)


# Pause execution until tasks reach Completed state.
wait_for_tasks_to_complete(batch_client,
                           jobs,
                           datetime.timedelta(hours=24))

print("  Success! All tasks reached the 'Completed' state within the "
      "specified timeout period.")

SyntaxError: invalid syntax (<ipython-input-11-390c58007a07>, line 87)

In [ ]:
grid_file_paths
#grid_file_names
grid_files


In [ ]:
user = batchmodels.UserIdentity(auto_user=batchmodels.AutoUserSpecification(elevation_level=batchmodels.ElevationLevel.admin, scope=batchmodels.AutoUserScope.task))

user

In [ ]:


#     # user = batchmodels.UserIdentity(auto_user=batchmodels.AutoUserSpecification(elevation_level=batchmodels.ElevationLevel.admin, scope=batchmodels.AutoUserScope.task))

#     tasks = []

#     for idx, input_file in enumerate(input_files):
#         filename = input_file_names[idx].split(".")[0]

#         command = [
#             "sudo bash",
#             "sudo docker run -it -d --name temp tgerrish/bhrad:latest",
#             "sudo docker cp $AZ_BATCH_TASK_WORKING_DIR/analysisGrid.json temp:/grid.json",
#             "sudo docker cp $AZ_BATCH_TASK_WORKING_DIR/sky_mtx.json temp:/sky_mtx.json",
#             "sudo docker cp $AZ_BATCH_TASK_WORKING_DIR/surfaces.json temp:/surfaces.json"
#             "sudo docker exec temp python RunHoneybeeRadiance.py -p ./grid.json -sm ./sky_mtx.json -s ./surfaces.json"
#             "sudo docker cp temp:/grid_result.json grid_result.json"
#             --filepath RunHoneybeeRadiance.py --blobname "runhbrad.py" --storageaccount "radfiles" --storagecontainer "0000000-jobname-3513" --sastoken "aRRVzOkO/kwS35CIwNVIa18aGoMfZD5D3yAy3GlorkkU2G+9q5rAscXoC21IIylJZerBefwMgxYYF3qzquALrw=="
            
#             'sudo docker run --rm -v $AZ_BATCH_TASK_WORKING_DIR:/usr/job:z antoinedao/radhoneywhale unzip /usr/job/{} -d /usr/job/'.format(input_file_names[idx]),
#                     'sudo docker run --rm -v $AZ_BATCH_TASK_WORKING_DIR:/usr/job antoinedao/radhoneywhale python3 /usr/convertToBash.py '
#                     '--filepath /usr/job',
#                     'sudo chmod +x $AZ_BATCH_TASK_WORKING_DIR/gridbased_annual/commands.sh',
#                     'sudo docker run --rm -v $AZ_BATCH_TASK_WORKING_DIR:/usr/job antoinedao/radhoneywhale /usr/job/gridbased_annual/commands.sh',
#                     'sudo docker run --rm -v $AZ_BATCH_TASK_WORKING_DIR:/usr/job antoinedao/radhoneywhale zip -r /usr/job/out_{} /usr/job/'.format(filename + ".zip"),
#                     'python $AZ_BATCH_NODE_SHARED_DIR/copyToBlob.py '
#                     '--filepath $AZ_BATCH_TASK_WORKING_DIR/{} --filename {} --storageaccount {} '
#                     '--storagecontainer {} --sastoken "{}"'.format(
#                      'out_'+filename + ".zip",
#                      'out_'+filename + ".zip",
#                      _STORAGE_ACCOUNT_NAME,
#                      output_container_name,
#                      output_container_sas_token)]

#         tasks.append(batch.models.TaskAddParameter(
#                 'Task_{}'.format(idx,filename),
#                 common.helpers.wrap_commands_in_shell('linux', command),
#                 resource_files=[input_file],
#                 user_identity=user
#                 )
#         )

#     batch_service_client.task.add_collection(job_id, tasks)

batch_service_client = batch_client
job_id = _JOB_ID + "_0"
input_files
input_file_names
output_container_name
output_container_sas_token
        print(filename)

In [ ]:
# COMMANDS FOR THE BATCH NODE

DOCKERIMAGE = "bhrad:latest"
IMAGENAME = "xyz"
PATHTOHOSTFILE_ANALYSISGRID = "C:/Users/tgerrish/Documents/GitHub/SAMAzure/TestFiles/Azure/radfiles/AnalysisGrids/zone1.json"
PATHTOHOSTFILE_SKYMATRIX = "C:/Users/tgerrish/Documents/GitHub/SAMAzure/TestFiles/Azure/radfiles/sky_mtx.json"
PATHTOHOSTFILE_SURFACES = "C:/Users/tgerrish/Documents/GitHub/SAMAzure/TestFiles/Azure/radfiles/surfaces.json"

# Start docker container
print("docker run --name {0:} {1:}".format(IMAGENAME, DOCKERIMAGE))

# Copy files from host to container
print("docker cp {0:} {1:}:/grid.json".format(PATHTOHOSTFILE_ANALYSISGRID, IMAGENAME))
print("docker cp {0:} {1:}:/sky_mtx.json".format(PATHTOHOSTFILE_SKYMATRIX, IMAGENAME))
print("docker cp {0:} {1:}:/surfaces.json".format(PATHTOHOSTFILE_SURFACES, IMAGENAME))

# Run Radiance command for daylight metrics processing
print("docker exec {0:} python RunHoneybeeRadiance.py -p ./grid.json -sm ./sky_mtx.json -s ./surfaces.json -o ./home".format(IMAGENAME))

# Copy result file back to host
print("docker cp {0:}:/home/grid_result.json grid_result.json".format(IMAGENAME))

print("\n\n")

print("docker run --name {0:} {1:} \\\n&& docker cp {2:} {0:}:/home/grid.json \\\n&& docker cp {3:} {0:}:/home/sky_mtx.json \\\n&& docker cp {4:} {0:}:/home/surfaces.json \\\n&& docker exec {0:} python _RunHoneybeeRadiance.py -p ./home/grid.json -sm ./home/sky_mtx.json -s ./home/surfaces.json -o ./home \\\n&& docker cp {0:}:/home/grid_result.json grid_result.json".format(IMAGENAME, DOCKERIMAGE, PATHTOHOSTFILE_ANALYSISGRID, PATHTOHOSTFILE_SKYMATRIX, PATHTOHOSTFILE_SURFACES))


In [ ]:
print("docker run --name {0:} {1:} && sleep 2 && docker cp {2:} {0:}:/home/grid.json && docker cp {3:} {0:}:/home/sky_mtx.json && docker cp {4:} {0:}:/home/surfaces.json && docker exec {0:} python _RunHoneybeeRadiance.py -p ./home/grid.json -sm ./home/sky_mtx.json -s ./home/surfaces.json -o ./home && docker cp {0:}:/home/grid_result.json grid_result.json".format(IMAGENAME, DOCKERIMAGE, PATHTOHOSTFILE_ANALYSISGRID, PATHTOHOSTFILE_SKYMATRIX, PATHTOHOSTFILE_SURFACES))